# Régime d'Assurance Chômage (RAC)

## Simulation RAC sans retraite anticipée

In [2]:
# Chargement des librairies
from cnaclib.rac import RAC
import pandas as pd
from datetime import datetime
import os

In [3]:
help(RAC)

Help on class RAC in module cnaclib.rac:

class RAC(builtins.object)
 |  RAC(DateRecrutement, DateCompression, SMM)
 |  
 |  REGIME ASSURANCE CHOMAGE : SIMULATEUR
 |  
 |  Cette Classe en 'python' permet de réaliser des simulations pour le calculs des différents éléments liés au régime d'assurance chômage.
 |  Elle permet de :
 |  - Vérifier la condition d'admission relative à l'experience professionnelle;
 |  - Calculer la durée de prise en charge (DPC);
 |  - Calculer le montant de la Contribution d'Ouverture de Droits;
 |  - Récupérer le montant du SNMG en fonction de la date;
 |  - Calculer les montants d'indemnités en fonction des 04 périodes;
 |  - Calculer les montants de cotisations de sécurité sociale (part patronale & part salariale );
 |  
 |  Parameters
 |  ----------
 |  
 |  DateRecrutement : date, 
 |      C'est de la date de recrutement du salarié chez le dernier employeur.
 |      Elle doit être exprimé selon le format : dd/mm/yyyy.
 |  
 |  
 |  DateCompression : date

In [21]:
# Déclaration des paramètres

dateRecrutement = '11/02/2015'
dateCompression =  '15/02/2021'
smm = float(60000)


In [22]:
# Instanciation de l'objet simulation
simul = RAC(dateRecrutement, dateCompression, smm) 

In [23]:
# Calcul de la durée d'experience
simul.Cal_Durexp()
dureeExperienceAnnee = simul.annee
dureeExperienceMois = simul.mois
dureeExperienceJours = simul.jours
dureeExperienceValidee = f"Votre expérience calculée est égale à {dureeExperienceAnnee} an(s), {dureeExperienceMois} moi(s) & {dureeExperienceJours} jour(s)"

print(dureeExperienceValidee)

Votre expérience calculée est égale à 6 an(s), 0 moi(s) & 4 jour(s)


In [24]:
# Vérification de l'admission
admission = simul.Verif_admission()
admissionMessage = simul.Message_admission()
print(admission)
print(admissionMessage)

Admis
Si vous remplissez les conditions prealablement citees et selon votre experience professionnelle calculee, vous pouvez beneficier du Regime Assurance Chomage.


In [40]:
# Calcul de la date d'admission (théorique)
dateAdmission = simul.Cal_DateAdmission()

print(dateAdmission.strftime("%d/%m/%Y"))

15/03/2021


In [26]:
# Calcul de la durée de prise en charge en mois
dpc = simul.Cal_DPC()

print(str(dpc))

13


In [27]:
# Calcul de la COD mensuel
codMensuel = simul.Cal_COD()[0]

print(codMensuel)

48000.0


In [28]:
# Calcul de la COD totale
codTotale= simul.Cal_COD()[1]

print(codTotale)

168000.0


In [33]:
# Renvoi des 04 periodes et les mois y afferents
MoisPeriode = simul.Cal_NumPeriode(dpc)

print(MoisPeriode)

{1: 'P1', 2: 'P1', 3: 'P1', 4: 'P2', 5: 'P2', 6: 'P2', 7: 'P3', 8: 'P3', 9: 'P3', 10: 'P4', 11: 'P4', 12: 'P4', 13: 'P4'}


In [36]:
# Renvoi le SNMG
snmg = simul.Cal_SNMG(dateAdmission)

print(snmg)

20000


In [37]:
# Renvoi le SNMG
SalRef = simul.Cal_SalRef(snmg)

print(SalRef)

40000.0


In [41]:
# Calcul les differents éléments du claendrier de paiement des indemnités
DateMois = simul.Cal_Indemnite(MoisPeriode, snmg, SalRef, dateAdmission)[0]
IndemniteBrut = simul.Cal_Indemnite(MoisPeriode, snmg, SalRef, dateAdmission)[1]
IndemniteNet = simul.Cal_Indemnite(MoisPeriode, snmg, SalRef, dateAdmission)[2]
PartPatronale = simul.Cal_Indemnite(MoisPeriode, snmg, SalRef, dateAdmission)[3]

print(DateMois)
print(IndemniteBrut)
print(IndemniteNet)
print(PartPatronale)

[datetime.datetime(2021, 4, 15, 0, 0), datetime.datetime(2021, 5, 15, 0, 0), datetime.datetime(2021, 6, 15, 0, 0), datetime.datetime(2021, 7, 15, 0, 0), datetime.datetime(2021, 8, 15, 0, 0), datetime.datetime(2021, 9, 15, 0, 0), datetime.datetime(2021, 10, 15, 0, 0), datetime.datetime(2021, 11, 15, 0, 0), datetime.datetime(2021, 12, 15, 0, 0), datetime.datetime(2022, 1, 15, 0, 0), datetime.datetime(2022, 2, 15, 0, 0), datetime.datetime(2022, 3, 15, 0, 0), datetime.datetime(2022, 4, 15, 0, 0)]
[40000.0, 40000.0, 40000.0, 32000.0, 32000.0, 32000.0, 24000.0, 24000.0, 24000.0, 20000.0, 20000.0, 20000.0, 20000.0]
[36600.0, 36600.0, 36600.0, 29280.0, 29280.0, 29280.0, 21960.0, 21960.0, 21960.0, 20000.0, 20000.0, 20000.0, 20000.0]
[3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0]


In [46]:
# Afficher le tableau relatif au calendrier de paiement des indemnités
tableau = simul.tableaux_Indemnites(MoisPeriode, DateMois, IndemniteBrut, IndemniteNet, PartPatronale)
tableau

,Periode,Mois,Date,Montant Indemnité Brut,Cotisation SS (PS),Montant Indemnité Net,Cotisation SS (PP)
0,P1,1,2021-04-15,40000.0,3400.0,36600.0,3000.0
1,P1,2,2021-05-15,40000.0,3400.0,36600.0,3000.0
2,P1,3,2021-06-15,40000.0,3400.0,36600.0,3000.0
3,P2,4,2021-07-15,32000.0,2720.0,29280.0,3000.0
4,P2,5,2021-08-15,32000.0,2720.0,29280.0,3000.0
5,P2,6,2021-09-15,32000.0,2720.0,29280.0,3000.0
6,P3,7,2021-10-15,24000.0,2040.0,21960.0,3000.0
7,P3,8,2021-11-15,24000.0,2040.0,21960.0,3000.0
8,P3,9,2021-12-15,24000.0,2040.0,21960.0,3000.0
9,P4,10,2022-01-15,20000.0,0.0,20000.0,3000.0


In [47]:
# Calcul de la date de fin de droit
dateFDD = simul.Cal_DateFDD(DateMois)

print(dateFDD.strftime("%d/%m/%Y"))

15/04/2022
